In [1]:
import nidaqmx

In [2]:
# Initializing nidaqmx system
system = nidaqmx.system.System.local()

In [3]:
# Printing out cards in chassis
for device in system.devices:
    print(device.product_type, device.name, device.serial_num)

PXIe-6341 PXI1Slot2 27399843
PXIe-6341 PXI1Slot4 28232070
PXIe-6341 PXI1Slot5 28232039
PXIe-6356 PXI1Slot6 29190299


In [4]:
for device in system.devices:
    print(device, device.do_max_rate)

Device(name=PXI1Slot2) 1000000.0
Device(name=PXI1Slot4) 1000000.0
Device(name=PXI1Slot5) 1000000.0
Device(name=PXI1Slot6) 10000000.0


In [5]:
# Streaming variable setup

from random import sample
from nidaqmx.constants import AcquisitionType, RegenerationMode
import numpy as np
import threading
import time

device = 'PXI1Slot6'
ao_channel = "ao1"
sample_rate = 1e6
buffer_seconds = 5
num_channels = 1
buffer_size = int(sample_rate * buffer_seconds)
half_buffer = buffer_size//2

In [6]:
# Waveform generation
def generate_waveform(start_idx, length):
    t = np.arange(start_idx, start_idx+length) / sample_rate
    return 1 * np.sign(np.sin(2 * np.pi * 10_000 * t))

def streaming_task(task, stop_event):
    next_idx = half_buffer
    while not stop_event.is_set():
        time.sleep(buffer_seconds/2)
        data = generate_waveform(next_idx, half_buffer)
        task.write(data, auto_start=False)
        next_idx += half_buffer

In [7]:
# Streaming a sine wave
with nidaqmx.Task() as ao_task:
    ao_task.ao_channels.add_ao_voltage_chan(f"{device}/{ao_channel}")

    ao_task.timing.cfg_samp_clk_timing(
        rate = sample_rate,
        sample_mode = AcquisitionType.CONTINUOUS,
        samps_per_chan=buffer_size
    )

    ao_task.out_stream.regen_mode = RegenerationMode.DONT_ALLOW_REGENERATION
    ao_task.out_stream.output_buf_size = buffer_size

    initial_data = generate_waveform(0, buffer_size)
    ao_task.write(initial_data)

    ao_task.start()

    stop_event = threading.Event()
    stream_thread = threading.Thread(target=streaming_task, args=(ao_task, stop_event))
    stream_thread.start()

    try:
        print('Streaming')
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Stopping")
        stop_event.set()
        stream_thread.join()
    finally:
        ao_task.stop()


Streaming
Stopping


In [ ]:
# Turning mirror on and off
device = 'PXI1Slot2'
ao_channel = "ao1"
with nidaqmx.Task() as ao_task:
    ao_task.ao_channels.add_ao_voltage_chan(f"{device}/{ao_channel}")
    ao_task.write(4.0)
    ao_task.start()
    time.sleep(5.0)
    ao_task.write(0)
    ao_task.stop()

Below is a successful implementation of waveform triggering at ~1MHz

In [8]:
import numpy as np
import nidaqmx
from nidaqmx.constants import AcquisitionType, Edge

fs = 2_000_000  # 1 MS/s
cycles = 63
samples_per_cycle = 2
num_samples = cycles * samples_per_cycle
waveform = np.tile([1.0, 0.0], cycles)


with nidaqmx.Task() as ao_task:
    ao_task.ao_channels.add_ao_voltage_chan("PXI1Slot6/ao1")
    ao_task.timing.cfg_samp_clk_timing(
        rate=fs,
        sample_mode=AcquisitionType.FINITE,
        samps_per_chan=num_samples
    )

    trigger_source = '/PXI1Slot2/PFI0' 
                                      
    ao_task.triggers.start_trigger.cfg_dig_edge_start_trig(
        trigger_source=trigger_source,
        trigger_edge=Edge.FALLING
    )

    ao_task.triggers.start_trigger.retriggerable = True

    ao_task.write(waveform)
    print(f"Waiting for trigger on {trigger_source} to output waveform...")
    ao_task.start()
    ao_task.wait_until_done(timeout=10.0)
    print("Waveform output complete.")
    ao_task.stop()

Waiting for trigger on /PXI1Slot2/PFI0 to output waveform...


KeyboardInterrupt: 

Need to try streaming data in and using this trigger to know when to get the next sample